<div>
    <h1><p style="text-align:left; font-size: 24px"> Análisis exploratorio acerca de los tweets relacionados al covit 19</p></h1>  
</div>

<img src="https://www.webespacio.com/wp-content/uploads/2010/12/wallpaper3-tipografia-twitter.jpg" align = 'center'>
<p><i>Presentado por: Veruska Palomino Cisneros</i></p>


<p style="text-align:justify; font-size: 20px">El presente trabajo busca realizar un análisis visual de los tweets que se han publicado durante la pandemia del covid 19. Siendo el estudio de caracter netamente educativo.</p>

<p p style="text-align:justify; font-size: 24px"> CONTENIDO </p>
<p style="text-align:justify; font-size: 18px">
    <ul style="text-align:justify; font-size: 18px">
        <li>Conjunto de datos</li>
        <li>Análisis del contenido de los Tweets</li>
        <li>Clusterización</li>
        <li>Visualización Geoespacial</li>
</ul>
    </p>
    

In [ ]:
# importación de librerias para realizar la lectura del conjunto de datos

import numpy as np
import pandas as pd
import plotly.express as px
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib import rcParams
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from iso3166 import countries
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot


rcParams['figure.figsize'] = 10,5.5
rcParams['figure.dpi'] = 60



In [ ]:
#cargamos el conjunto de datos en un dataframe y mostramos los 10 primeros registros

df = pd.read_csv('/kaggle/input/covid19-tweets/covid19_tweets.csv')
df.head(10)

In [ ]:
#Observamos que columnas tiene nuestro conjunto de datos y los tipos de cada dato
df.info()

<p style="text-align:justify; font-size: 24px"> Visualización de Datos </p>

<p style="text-align:justify; font-size: 18px">1.- Usuarios por número de Tweet </p>
    

In [ ]:
number_tweets = df['user_name'].value_counts().reset_index()
number_tweets.columns = ['user_name', 'tweets']


sns.barplot(x = "tweets", y = "user_name", data = number_tweets.head(20), palette='PuOr')
plt.show()

<p style="text-align:justify; font-size: 18px">
2.- Número de Tweets por seguidores</p>

In [ ]:
top_users = df.sort_values('user_followers', ascending =  False).drop_duplicates(subset = 'user_name', keep = 'first')
top_users = top_users[['user_name', 'user_followers']]
top_users = pd.merge(top_users, number_tweets, 'inner')


#Normalize the scale to make the color bar on the right of the bar plot
norm = plt.Normalize(top_users['tweets'].min(), top_users['tweets'].max())
sm = plt.cm.ScalarMappable(cmap="PuOr", norm=norm)
sm.set_array([])
#Show the barplot with color bar
ax = sns.barplot(x="user_followers", y = "user_name" , data = top_users.head(15), hue = 'tweets', dodge = False, palette = 'PuOr')
ax.get_legend().remove()
ax.figure.colorbar(sm)
plt.show()


<p style="text-align:justify; font-size: 18px">
En el gráfico analizado en el punto 2, podemos observar que el usuario CNN cuenta con una mayor cantidad de seguidores, pero estos seguidores no han muchos comentarios sobre el covid 19, a diferencia de los usuarios The Times of India y China Xinhua que han realizadop una mayor cantidad de comentarios pero que no cuentan con una gran cantidad de seguidores.</p>

<p style="text-align:justify; font-size: 18px">
3.- Medios más usados para publicar Tweets.</p>
    

In [ ]:
device = df['source'].value_counts().reset_index()
device.columns = ['source', 'count']
device['percent_tweets'] = round(device['count']/device['count'].sum()*100, 2)


sns.barplot(x = "percent_tweets", y = "source", data = device.head(10), palette = 'PuOr')
plt.show()

<p style="text-align:justify; font-size: 18px">
El gráfico del apartado 3 muestra claramente que los 3 dispositivos más usados para publicar tweets son Twitter apps, Twitter Android y Twitter for iphone. En su contraste entre las menos uasadas tenemos a Sprout Social, Instragam y FTTT. 
    </p>
<p style="text-align:justify; font-size: 18px">
4.- Incremento de Tweets por Año
</p>


In [ ]:
df['user_created'] = pd.to_datetime(df['user_created'])
new_users = df[['user_created', 'user_name']].drop_duplicates(subset = 'user_name', keep = 'first')
new_users['user_created']= new_users['user_created'].dt.year
count_year = new_users['user_created'].value_counts().reset_index()
count_year.columns = ['year', 'number']
count_year

count_year['year'] = count_year[count_year['year']>1990]
sns.lineplot(x = 'year', y = 'number', data = count_year, hue="year",marker = 'o')
plt.xlabel('Year')
plt.ylabel('Number of New Users')
plt.show()


<p style="text-align:justify; font-size: 18px">
El gráfico del apartado 4 muestra claramente el incremento o decremento de usuarios en twitter entre los años 2006 y 2020, se puede observar un incremento de abismal de usuarios en el año 2009 que rápidamente a ido en descenso en los años posteriores. Entre los años 2014 y 2018 se mantiene una constante en la cantidad de usuarios creados y a partir de finales del 2018 la tendencia aumenta hasta el 2020. </p>

<p style="text-align:justify; font-size: 24px"> Análisis de Textos </p>
<p style="text-align:justify; font-size: 18px">
A continuación realizaremos un análisis del texto que contienen los tweets que se han realizado durante la pandemia del covid 19. Para ello usaremos la columna hashtag para poder crear un wordcloud.</p>

In [ ]:
df['hashtags'] = df['hashtags'].fillna('[]')
df['hashtags_count'] = df['hashtags'].apply(lambda x: len(x.split(',')))
df.loc[df['hashtags'] == '[]', 'hashtags_count'] = 0

# let's see the number of hashtags used by users
#hashtag_per_user = df[['user_name','hashtags_count']].sort_values('hashtags_count', ascending =  False).drop_duplicates(subset = 'user_name', keep = 'first')
#sns.barplot(x="hashtags_count", y = "user_name", data = hashtag_per_user.head(30), palette = 'Blues_r')
#plt.show()

<p style="text-align:justify; font-size: 18px">
Lo primero quew haremos será reemplazar los datos nulos para poder obtener mejores gráficas y determinar cuales son los hashtags más populares en la base.</p>

In [ ]:
def hashtags_split(x):
    return str(x).lower().replace('[','').replace(']','').replace("'",'').replace(" ", '').split(',')

hashtag_tweets = df.copy()
hashtag_tweets['hashtag'] = hashtag_tweets['hashtags'].apply(lambda row: hashtags_split(row))
hashtag_tweets = hashtag_tweets.explode('hashtag')
hashtag_tweets.loc[hashtag_tweets['hashtag'] == '', 'hashtag'] = 'No Hashtag'
hashtag_tweets.head()

<p style="text-align:justify; font-size: 18px">
El presente gráfico muestra cuales son los Hastags más relevantes en la base de datos</p>

In [ ]:
hashtag_number = hashtag_tweets['hashtag'].value_counts().reset_index()
hashtag_number.columns = ['hashtag', 'count']

sns.barplot(x="count", y = "hashtag", data = hashtag_number.head(10), palette = 'PuOr')
plt.show()

<p style="text-align:justify; font-size: 18px">
   Ahora procederemos a realizar un wordcloud de los tópicos más relevantes.
    </p>

In [ ]:
text = "".join(tweet for tweet in df['text'])
stopwords = set(STOPWORDS)
stopwords.update(['https', 't','co', 'many', 's'])

wordcloud = WordCloud(stopwords=stopwords, background_color='black').generate(text)

plt.imshow(wordcloud)
plt.axis('off')
plt.show()

<p style="text-align:justify; font-size: 18px">
    Claramente podemos observar que las palabras que más se repiten son COVID 19, COVID, CORONAVIRUS, CASE, y people.
    </p>

<p style="text-align:justify; font-size: 24px">
Análisis de Sentimientos.</p>

In [ ]:
vec = TfidfVectorizer(stop_words = 'english')
vec.fit(df['text'].values)
features = vec.transform(df['text'].values)

In [ ]:
kmeans = KMeans(n_clusters = 3, random_state = 0)
kmeans.fit(features)

In [ ]:
res = kmeans.predict(features)
df['Cluster'] = res


In [ ]:
text_cluster_1 = " ".join(tweet for tweet in df[df['Cluster'] == 0]['text'])
stopwords = set(STOPWORDS)
stopwords.update(['https', 't','co', 'many', 's'])

wordcloud_1 = WordCloud(max_words = 250, stopwords=stopwords, background_color='black').generate(text_cluster_1)

plt.imshow(wordcloud_1)
plt.axis('off')
plt.show()

In [ ]:
text_cluster_2 = " ".join(tweet for tweet in df[df['Cluster'] == 1]['text'])
stopwords = set(STOPWORDS)
stopwords.update(['https', 't','co', 'many', 's'])

wordcloud_2 = WordCloud(max_words = 250, stopwords=stopwords, background_color='black').generate(text_cluster_2)

plt.imshow(wordcloud_2)
plt.axis('off')
plt.show()

In [ ]:
text_cluster_3 = " ".join(tweet for tweet in df[df['Cluster'] == 1]['text'])
stopwords = set(STOPWORDS)
stopwords.update(['https', 't','co', 'many', 's'])

wordcloud_3 = WordCloud(max_words = 250, stopwords=stopwords, background_color='black').generate(text_cluster_3)

plt.imshow(wordcloud_3)
plt.axis('off')
plt.show()

<p style="text-align:justify; font-size: 18px">
    De los 3 grupos que se tienen para poder analizar se puede observar la siguiente el primer grupo muestra una tendencia a comentarios negativos en relación al covit destacan palabras como pandemia, auxilio, casos y coronavirus. Sin embargo mientras analizamos el clustr 2 y 3 esta tendencia dismuniye pues está más destinada a la prevención con palabras como mascarilla, noticias de hoy y el mundo por lo tanto podemos afirmnar que es un escenario mas positivo. 
    </p>
